In [1]:
import valenspy as vp
import xarray as xr
from pathlib import Path

import dask

## The Input Manager
The input manager aims to make accessing shared standard datasets easy.
As the available data, data path and variables is HPC specific, the input manager only works for HPC systems specified in the dataset_PATHS.yml. To add a dataset or a new machine, add your machine name, datasets and paths to the dataset_PATHS.yml file.

In [2]:
manager = vp.InputManager(machine='hortense')

### Usage

For everyday use, a dataset can be accessed through the load data functionality. Variables are accessed through their CORDEX variable name. 

You do not need to know the how the variable is called in the oroginal dataset. E.g. In era5, the 2m temperature is called 'tp' but is accessed here through the CORDEX variable name 'tas'. The original name is added to the attributes of the variable for reference.

Note that the files that are found and used to load the data are printed and the CF_status of the ds is printed. This is to help debug if the data is not loaded as expected.


In [7]:
ds = manager.load_data("ERA5",["pr"], period=[2000,2001],freq="daily",region="europe", path_identifiers=["min"])
ds

File paths found:
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/total_precipitation/era5-daily_min-europe-total_precipitation-2000.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/total_precipitation/era5-daily_min-europe-total_precipitation-2001.nc
The file is ValEnsPy CF compliant.
50.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['pr']
Unknown to ValEnsPy: ['time_bnds']


<xarray.Dataset> Size: 276MB
Dimensions:    (time: 731, bnds: 2, lat: 163, lon: 289)
Coordinates:
  * time       (time) datetime64[ns] 6kB 2000-01-01T11:30:00 ... 2001-12-31T1...
  * lon        (lon) float32 1kB -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * lat        (lat) float32 652B 73.5 73.25 73.0 72.75 ... 33.5 33.25 33.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 12kB dask.array<chunksize=(366, 2), meta=np.ndarray>
    pr         (time, lat, lon) float64 275MB dask.array<chunksize=(362, 161, 286), meta=np.ndarray>
Attributes:
    CDI:               Climate Data Interface version 1.9.5 (http://mpimet.mp...
    Conventions:       CF-1.6
    history:           Tue Feb 08 13:46:56 2022: cdo -z zip daymin /mnt/HDS_B...
    frequency:         day
    CDO:               Climate Data Operators version 1.9.5 (http://mpimet.mp...
    dataset:           ERA5
    period:            [2000, 2001]
    freq:              daily
    path_identifiers:  ['min']
    region:            europe

Note that depending on your search criteria, metadata is added to the dataset.

With the load_dataset functionalilty you can also:
- load multiple variables simultaneously and/or
- **not** convert the ds to cf-compliant format and/or ``cf_convert=False``
- Add additional meta_data to the ds - if using the 

However, the ds is then not in cf convention and applying diagnostics will not work.

In [10]:
ds = manager.load_data("EOBS",["tas","pr"], path_identifiers=["mean"], cf_convert=True, metadata_info={"creator":"ME"})
ds

File paths found:
/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/rr_ens_mean_0.1deg_reg_v29.0e.nc
/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_mean_0.1deg_reg_v29.0e.nc
The file is ValEnsPy CF compliant.
100.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['pr', 'tas']


<xarray.Dataset> Size: 283GB
Dimensions:  (lat: 930, lon: 1409, time: 27028)
Coordinates:
  * lat      (lat) float64 7kB 25.05 25.05 25.15 25.15 ... 71.35 71.45 71.45
  * lon      (lon) float64 11kB -24.95 -24.95 -24.85 ... 45.35 45.45 45.45
  * time     (time) datetime64[ns] 216kB 1950-01-01 1950-01-02 ... 2023-12-31
Data variables:
    pr       (time, lat, lon) float32 142GB dask.array<chunksize=(102, 930, 1409), meta=np.ndarray>
    tas      (time, lat, lon) float32 142GB dask.array<chunksize=(102, 930, 1409), meta=np.ndarray>
Attributes:
    E-OBS_version:     29.0e
    Conventions:       CF-1.4
    References:        http://surfobs.climate.copernicus.eu/dataaccess/access...
    history:           Thu Feb 22 11:30:38 2024: ncks --no-abc -d time,0,2702...
    NCO:               netCDF Operators version 5.1.4 (Homepage = http://nco....
    dataset:           EOBS
    creator:           ME
    path_identifiers:  ['mean']

The name of the dataset (e.g. "ERA5") should be in the dataset_PATHS.yml file to be able to find the data. If using the inputconvertor ``cf_convert=True`` (default option) a corresponding input_convertor should be available. Currently the following datasets have input convertors:

In [5]:
vp.inputconverter.INPUT_CONVERTORS

{'ERA5': <valenspy.inputconverter.InputConverter at 0x15393bd24070>,
 'EOBS': <valenspy.inputconverter.InputConverter at 0x15393bd240a0>}

## A peak inside the manager

The input_manager uses the path specified in the dataset_PATHS.yml for the given dataset and machine to search all .nc files and file paths that match the filtering requested. The following function is doing all the "magic":

In [6]:
manager._get_file_paths("EOBS",["tas","pr"], path_identifiers=["mean"]) #The magic happens here ! All

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/rr_ens_mean_0.1deg_reg_v29.0e.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_mean_0.1deg_reg_v29.0e.nc')]

Above all paths starting with '/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/' and containing the original name (long or short name) for 'tas' or 'pr' in this case tg and rr and "mean" are selected. Other options are:
- region:
- period: [start_year, end_year] possibly more is covered (note some datasets are not stored by year)!
- frequency: 
- other: Any other keywords to filter by are specified in the path_identifier. E.g. 'mean' for monthly mean data.

For more information see the documentation on the input_manager and the load_data function.